# Active Transporation Program - State Component Report

This report is an anlysis of the Active Transportation Program Application Cycles. Currently, this analysis is grouped at the *cycle* level. 

In [1]:
import warnings
warnings.filterwarnings('ignore')


In [2]:
%%capture
import intake
import numpy as np
import pandas as pd
import geopandas as gpd
from calitp import to_snakecase
from dla_utils import _dla_utils
from IPython.display import HTML, Markdown
from siuba import *
from shared_utils import geography_utils
from shared_utils import calitp_color_palette as cp
from shared_utils import styleguide

import altair as alt

import _data_cleaning
import _report_utils

In [3]:
import fiona

from ipyleaflet import Map, GeoJSON, projections, basemaps, GeoData, LayersControl, WidgetControl, GeoJSON, LegendControl
from ipywidgets import Text, HTML

In [4]:
GCS_FILE_PATH = 'gs://calitp-analytics-data/data-analyses/dla/atp/'


In [5]:
## read in data

In [ ]:
df = _report_utils.read_in_joined_data()

In [ ]:
df = (_report_utils.reorder_namecol(df,
                    og_name_col= 'a1_imp_agcy_name',
                    new_name_col= 'imp_agency_name_new', 
                    split_on = ", ",
                   order_on ='pt2_pt1'))

In [ ]:
pd.set_option("display.max_columns",500)

In [ ]:
## paramaters cell -- not using 
#district = 4

In [ ]:
## parameters cell
cycle = 5

In [ ]:
## subset df to just the district
#df = df_all>>filter(_.a2_ct_dist==district)

In [ ]:
df = df>>filter(_.project_cycle==cycle)

In [ ]:
## df with correct mapping coordinates
df_map = _report_utils.prep_mapping_data(df)

In [ ]:
## from amanda's notebook for Project prioritization: https://github.com/cal-itp/data-analyses/blob/main/project_prioritization/_utils.py
shapes = "https://gis.data.ca.gov/datasets/0144574f750f4ccc88749004aca6eb0c_0.geojson?outSR=%7B%22latestWkid%22%3A3857%2C%22wkid%22%3A102100%7D"
district_shapes = gpd.read_file(f"{shapes}").to_crs(epsg=4326)  

## df with ct district level shapes
by_dist = _report_utils.nunique_by_geography(df_map,
                         district_shapes,
                         groupby_cols = ['a2_ct_dist', 'data_origin'],
                         agg_col=  'project_app_id',
                         geodf_merge_on_col = 'DISTRICT',
                         sum_merge_on_col = 'a2_ct_dist')

In [ ]:
## long df with various district application counts (assembly, senate, congressional, caltrans)
districts = _report_utils.get_district_level_data(by_dist, df_map)

In [ ]:
funded = df>>filter(_.awarded=='Y')

In [ ]:
display(HTML(f"<h2>Quick Look at Cycle {cycle}</h2>"))

display(HTML(f"Out of {len(df)} Active Transportation Program Project Applications, "
            f"there are <strong>{(df>>filter(_.awarded=='Y')).project_app_id.nunique()} "
            f"projects</strong> that recieved funding over "
            f"{df.project_cycle.nunique()} cycles."))


In [ ]:
display(HTML(f"<h4> What were the application outcomes "
             # f"for District {district} "
             f"in Cycle {cycle} across the State?</h4>"))
display(HTML(_dla_utils.pretify_tables((df>>count(_.data_origin)))))

In [ ]:
display(HTML(f'The total dollar amount awarded in Cycle {cycle} for the Statewide Component was '
             f'<strong> {("${:,.2f}".format((funded["total_atp_$"].sum())))} </strong>'))

In [ ]:
by_dist = by_dist.rename(columns= {'a2_ct_dist':'Caltrans District'})

In [ ]:
ct_dist_sr = by_dist>>select(_['Caltrans District'], _['Success Rate'])
ct_dist_sr['Success Rate'] = ct_dist_sr['Success Rate'].transform(lambda x: '{:,.2%}'.format(x))


In [ ]:
display(HTML(f"<h4>Caltrans District Success rate in Cycle {cycle}</h4>"))

display(HTML(_dla_utils.pretify_tables(ct_dist_sr)))

In [ ]:
count_proj_ct = (by_dist >> select(_['Caltrans District'], _.Application, _.Funded)
 >> gather('Application Status', 'Number of Projects', _.Application, _.Funded))>>arrange(_['Caltrans District'])


In [ ]:
ct_totals = (alt.Chart(count_proj_ct).mark_bar(size=30).encode(
    x='Caltrans District',
    y='Number of Projects',
    color='Application Status',
     tooltip=['Caltrans District','Application Status', 'Number of Projects']).properties(
    width=600,
    height=300))


In [ ]:
display(HTML(f"<h4> Application Status by Caltrans District in Cycle {cycle}</h4>"))

ct_totals

## District Maps
* Maps at the Assembly, Congressional, Senate, and Caltrans District Levels

In [ ]:
import folium

In [ ]:
display(HTML("<h3>Number of Applications by County</h3>"))

In [ ]:
m = by_dist.explore(
     column="Application", 
     legend=True,
    cmap = 'Oranges',
     legend_kwds=dict(colorbar=True),
     tooltip=["DISTRICT", "Application"], 
     name="Number of Applications by County")

df_map.explore(
     m=m, 
     color="data_origin", 
     marker_kwds=dict(radius=3, fill=True),
     tooltip=["data_origin", "project_cycle", "imp_agency_name_new", "a1_imp_agcy_city", "a2_info_proj_name"], 
     tooltip_kwds=dict(labels=True),
     name="Project Locations"
)

folium.TileLayer('Stamen Toner', control=True).add_to(m) 
folium.LayerControl().add_to(m)

m

In [ ]:
display(HTML("<h3>Mapping Option for Assembly Districts (or others)</h3>"))

In [ ]:
_report_utils.map_districts(districts, 
                 district_type= "Assembly District",
                 district_type_col= "District Type",
                 mapping_col= "Success Rate",
              tool_tip_cols= ['District Type', 'District', 'Success Rate'])

In [ ]:
#display(HTML("<h3>Mapping Option for All Districts (dropdown)</h3>"))

In [ ]:
#_report_utils.add_dropdown_map(districts)

In [ ]:
## Metrics

In [ ]:
# unique_agencies = df>>group_by(_.project_cycle, _.a2_county, _.data_origin)>>summarize(n_unique_agency= _.imp_agency_name_new.nunique())

In [ ]:
# unique_agencies =(spread(unique_agencies, "data_origin", "n_unique_agency"))

In [ ]:
# unique_agencies = unique_agencies.rename(columns={"a2_county":"county_name"})

In [ ]:
# unique_agencies['Application'].fillna(0, inplace=True)
# unique_agencies['Funded'].fillna(0, inplace=True)

In [ ]:
# unique_agencies['Application'] = unique_agencies['Application'].astype('int32')
# unique_agencies['Funded'] = unique_agencies['Funded'].astype('int32')

In [ ]:
# display(HTML("<h3>Number of Unique Agencies By County</h3>"))
# display(HTML(_dla_utils.pretify_tables(unique_agencies)))


## Application and Funded Project Details by District

In [ ]:
from shared_utils import altair_utils

### District 1 - Eureka

In [ ]:
_report_utils.map_dist_proj(funded, df, df_map, 1)

### District 2 - Redding

In [ ]:
_report_utils.map_dist_proj(funded, df, df_map, 2)

### District 3 - Marysville

In [ ]:
_report_utils.map_dist_proj(funded, df, df_map, 3)

In [ ]:
#df>>filter(_.a1_imp_agcy_name=="5002")

In [ ]:
#df>>filter(_.a1_imp_agcy_name=="5447")

### District 4 - Oakland 

In [ ]:
_report_utils.map_dist_proj(funded, df, df_map, 4)

In [ ]:
d4 = df_map>>filter(_.a2_ct_dist==4)

In [ ]:
display(HTML(f"Out of {len(d4)} projects in District {d4['a2_ct_dist'].iloc[0]}, "
             f"{len(d4>>filter(_.awarded=='Y'))} were funded"))

### District 5 - San Luis Obispo

In [ ]:
_report_utils.map_dist_proj(funded, df, df_map, 5)

### District 6 - Frenso

In [ ]:
_report_utils.map_dist_proj(funded, df, df_map, 6)

### District 7 - Los Angeles

In [ ]:
_report_utils.map_dist_proj(funded, df, df_map, 7)

### District 8 - San Bernardino

In [ ]:
_report_utils.map_dist_proj(funded, df, df_map, 8)

### District 9 - Bishop

In [ ]:
_report_utils.map_dist_proj(funded, df, df_map, 9)

### District 10 - Stockton

In [ ]:
_report_utils.map_dist_proj(funded, df, df_map, 10)

### District 11 - San Diego

In [ ]:
_report_utils.map_dist_proj(funded, df, df_map, 11)

### District 12 - Irvine

In [ ]:
_report_utils.map_dist_proj(funded, df, df_map, 12)